In [1]:
#Aktivacija biblioteka za grafičku i numeričku obradu podataka
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt


KeyboardInterrupt



In [ ]:
#Freezing sampling order and importing the dataset 
seed_value= 100 #random seed number
import random 
random.seed(seed_value) 
bp = pd.read_excel('Database source') #specify path of your database file

In [ ]:
#Overview of the database
bp.head() 

In [ ]:
#Graphical presentation of the relationships between variables of the dataset
sb.pairplot(bp)

In [ ]:
bp.describe() #important informations about the distributions of the dataset variables

In [ ]:
#Dataset train/test splitting strategy
from sklearn.model_selection import train_test_split 

In [ ]:
X = bp[['NoSt','NoSp','LoSp']].values #input samples (BARE frames)

In [ ]:
y = bp['TFP'].values #output samples (BARE frames)

In [ ]:
#Train-test-split
np.random.seed(seed_value) #freezing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=seed_value)

In [ ]:
#Data preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

In [ ]:
scaling = MinMaxScaler(feature_range=(-1, 1))

In [ ]:
X_tn=scaling.fit(X_train)

In [ ]:
bpformula = scaling.transform(bpformula)

In [ ]:
X = scaling.transform(X) #complete input data

In [ ]:
X_train = scaling.transform(X_train)

In [ ]:
X_test = scaling.transform(X_test)

In [ ]:
y_train=np.reshape(y_train,(1478,1)) #depends on the train/test splitting strategy
y=np.reshape(y,(1848,1)) #depends on the train/test splitting strategy

scaling.fit(y_train)
y_train = scaling.transform(y_train)

In [ ]:
y_test=np.reshape(y_test,(370,1)) #depends on the train/test splitting strategy
y_test = scaling.transform(y_test)

In [ ]:
y = np.reshape(y,(1848,1)) #depends on the train/test splitting strategy
y = scaling.transform(y)

In [ ]:
#Activation of TensorFlow and Sklearn libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout

In [ ]:
tf.random.set_seed(seed_value)

In [ ]:
#ANN architecture
model=Sequential()
model.add(Dense(4,activation='tanh'))
model.add(Dense(1,activation='linear'))

#ANN algorithm/optimizer - Adam optimizer / SGD optimizer
opt = tf.keras.optimizers.Adam(learning_rate=0.001,beta_1=0.9,beta_2=0.999,epsilon=1e-08)
model.compile(optimizer=opt,loss='mse',steps_per_execution=5,run_eagerly=False)

In [ ]:
#Implementation of early stopping rule
from tensorflow.keras.callbacks import EarlyStopping

In [ ]:
early_stopping = EarlyStopping(monitor='val_loss',mode='min',patience=100, restore_best_weights=True)

In [ ]:
#Fitting the model
model.fit(x=X_train,y=y_train,epochs=1000,validation_data=(X_test,y_test),shuffle=True,callbacks=[early_stopping])

In [ ]:
error_bp = pd.DataFrame(model.history.history) #monitoring error values

In [ ]:
error_bp

In [ ]:
error_bp.plot()
fig = error_bp.plot(grid='default',title='Loss / Iterations',xlabel='Epochs',ylabel='Mean Squared Error').get_figure()

In [ ]:
#Model evaluation
model.evaluate(X_train,y_train)
model.evaluate(X_test,y_test)

In [ ]:
#Comparison between dataset and ANN results
test_prediction = model.predict(X_test)
training_prediction = model.predict(X_train)
all_prediction = model. predict(X)

X_train = scaling.inverse_transform(X_train) #POVRATAK NA ORIGINALNE VREDNOSTI
y_train = scaling.inverse_transform(y_train)
X_test = scaling.inverse_transform(X_test)
y_test = scaling.inverse_transform(y_test)

X = scaling.inverse_transform(X)
y = scaling.inverse_transform(y)
test_prediction = scaling.inverse_transform(test_prediction)
training_prediction = scaling.inverse_transform(training_prediction)
all_prediction = scaling.inverse_transform(all_prediction)

In [ ]:
test_prediction = pd.DataFrame(test_prediction.reshape(370,)) #depends on the train/test splitting strategy
training_prediction = pd.DataFrame(training_prediction.reshape(1478,)) #depends on the train/test splitting strategy
all_prediction = pd.DataFrame(all_prediction.reshape(1848,)) #depends on the train/test splitting strategy

In [ ]:
test_prediction

In [ ]:
training_prediction

In [ ]:
all_prediction

In [ ]:
#Poređenje rezultata na test uzorku zbog evaluacije mreže
prediction_bp = pd.DataFrame(y_test,columns=['Output dataset'])
prediction_bptraining = pd.DataFrame(y_train,columns=['Output dataset'])
predikcija_bpsve = pd.DataFrame(y,columns=['Output dataset'])

In [ ]:
prediction_bp = pd.concat([prediction_bp,test_prediction],axis=1)
prediction_bptraining = pd.concat([prediction_bptraining,training_prediction],axis=1)
predikcija_bpsve = pd.concat([predikcija_bpsve,all_prediction],axis=1)

In [ ]:
#Promena imena kolona

prediction_bp.columns=['Output dataset','Output ANN']

prediction_bptraining.columns=['Output dataset','Output ANN']

prediction_bpall.columns=['Output dataset','Output ANN']

In [ ]:
#Plotting the results
sb.regplot(x='Output ANN',y='Output dataset',ci=None, scatter=True, data=prediction_bp)

In [ ]:
from sklearn import metrics
R_square= metrics.r2_score(y_test,test_prediction)

In [ ]:
sb.regplot(x='Output ANN',y='Output dataset',ci=None, scatter=True, data=prediction_bptraining)

In [ ]:
R_square= metrics.r2_score(y_train,training_prediction)

In [ ]:
sb.regplot(x='Output ANN',y='Output dataset',ci=None, scatter=True, data=prediction_bpall)

In [ ]:
R_square= metrics.r2_score(y,all_prediction)

In [ ]:
weights = model.get_weights() #ANN parameters

In [ ]:
weights